In [116]:
#####Probit based Hierarchical Matrix Factorization####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [117]:
####任意の相関行列(分散共分散行列)を作成する関数####
##任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = (upper - lower) *rand(col*col) + lower   #相関係数の乱数ベクトルを作成
    rho = np.reshape(np.array(cov_vec), (col, col)) * np.tri(col)   #乱数ベクトルを下三角行列化
    Sigma = np.diag(np.diag(rho + rho.T) + 1) - (rho + rho.T)   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = scipy.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(eigen_val.shape[0]-1):
        if eigen_val[i] < 0:
            eigen_val[i] = (eigen_upper - eigen_lower) * rand(1) + eigen_lower
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(pow(np.diag(Sigma), 0.5)[:, np.newaxis], pow(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

In [118]:
##相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

In [119]:
####データの発生####
##データの設定
k = 10   #基底数
hh = 10000   #ユーザー数
item = 3000   #アイテム数
pt = np.random.poisson(np.random.gamma(27.5, 1/0.3, hh), hh)   #購買接触数
hhpt = np.sum(pt)   #総サンプル数

In [120]:
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    pt_id[user_id==i] = range(pt[i])

In [121]:
#インデックスの作成
index = np.array(range(hhpt))
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]

In [122]:
##説明変数の生成
##素性ベクトルを生成
k1 = 2; k2 = 3; k3 = 4
x1 = np.array(np.random.random(hhpt*k1)).reshape((hhpt, k1))
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hhpt)
x3 = np.delete(x3, np.argmin(np.sum(x3, axis=0)), axis=1)   #冗長な変数の削除
X = np.concatenate((np.repeat(1, hhpt)[:, np.newaxis], x1, x2, x3), axis=1)
column = X.shape[1]

In [123]:
##階層モデルの説明変数を生成
#ユーザーの説明変数を生成
k1 = 1; k2 = 3; k3 = 5
u1 = np.array(np.random.random(hh*k1)).reshape((hh, k1))
u2 = np.zeros((hh, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    u2[:, j] = np.random.binomial(1, prob, hh)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hh)
u3 = np.delete(u3, np.argmin(np.sum(u3, axis=0)), axis=1)   #冗長な変数の削除
u = np.concatenate((np.repeat(1, hh)[:, np.newaxis], u1, u2, u3), axis=1)
column_u = u.shape[1]

#アイテムの説明変数を生成
k1 = 2; k2 = 3; k3 = 4
v1 = np.array(np.random.random(item*k1)).reshape((item, k1))
v2 = np.zeros((item, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    v2[:, j] = np.random.binomial(1, prob, item)
v3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), item)
v3 = np.delete(v3, np.argmin(np.sum(v3, axis=0)), axis=1)   #冗長な変数の削除
v = np.concatenate((np.repeat(1, item)[:, np.newaxis], v1, v2, v3), axis=1)
column_v = v.shape[1]

In [124]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[user_index[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [125]:
#アイテムインデックスを作成
index = np.array(range(hhpt))
item_index = [j for j in range(item)]
for j in range(item):
    item_index[j] = index[item_id==j]

In [126]:
####応答変数の生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)

    ##階層モデルのパラメータの設定
    #階層モデルの分散を設定
    Cov = np.diag(uniform(0.01, 0.15, column))
    Cov_u = np.diag(uniform(0.01, 0.2, k))
    Cov_v = np.diag(uniform(0.01, 0.2, k))
    Covt = Cov; Cov_ut = Cov_u; Cov_vt = Cov_v
    
    #階層モデルの回帰係数を設定
    alpha = np.hstack((np.random.normal(-0.2, 0.5, column_u).reshape(column_u, 1), 
                       np.random.normal(0, 0.5, (column-1)*column_u).reshape(column_u, column-1)))
    alpha_u = np.random.normal(0, 0.5, k*column_u).reshape(column_u, k)
    alpha_v = np.random.normal(0, 0.5, k*column_v).reshape(column_v, k)
    alphat = alpha; alpha_ut = alpha_u; alpha_vt = alpha_v

    ##モデルパラメータを生成
    #素性ベクトルと行列分解のパラメータを生成
    beta = np.dot(u, alpha) + np.random.multivariate_normal(np.repeat(0, column), Cov, hh)
    theta_u = np.dot(u, alpha_u) + np.random.multivariate_normal(np.repeat(0, k), Cov_u, hh)
    theta_v = np.dot(v, alpha_v) + np.random.multivariate_normal(np.repeat(0, k), Cov_v, item)
    betat = beta; theta_ut = theta_u; theta_vt = theta_v         
                                                        
    #標準偏差を設定
    Sigma = 1

    ##潜在効用から応答変数を生成
    #回帰モデルの平均構造
    W = theta_u[user_id, :]
    H = theta_v[item_id, :]
    mu = np.dot(X * beta[user_id, :], np.repeat(1, column)) + np.dot(W * H, np.repeat(1, k))

    #正規分布から潜在効用を生成
    U = np.random.normal(mu, Sigma, hhpt); UT = U   #潜在効用
    y = np.array((U > 0), dtype='int')   #潜在効用を購買ベクトルに変換
    
    if((np.mean(y) > 0.2) & (np.mean(y) < 0.4)):
        break

1


In [127]:
####マルコフ連鎖モンテカルロ法でProbit base hierarchical Matrix Factorizationを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [128]:
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [129]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 500/keep
disp = 10

In [130]:
##事前分布の設定
#階層モデルの事前分布
Deltabar = np.zeros((column_u, column))
Deltabar_u = np.zeros((column_u, k))
Deltabar_v = np.zeros((column_v, k))
ADelta = ADelta_u =0.01 * np.diag(np.ones(column_u))
ADelta_v = 0.01 * np.diag(np.ones(column_v))
nu = nu1 = nu2 = 1
V = 0.1 * np.diag(np.ones(column))
V1 = 0.1 * np.diag(np.ones(k))
V2 = 0.1 * np.diag(np.ones(k))

In [154]:
##パラメータの真値
#階層モデルのパラメータ
Cov = Covt; Cov_inv = np.linalg.inv(Cov)
Cov_u = Cov_ut; Cov_u_inv = np.linalg.inv(Cov_u)
Cov_v = Cov_vt; Cov_v_inv = np.linalg.inv(Cov_v)
alpha = alphat; alpha_mu = np.dot(u, alpha)
alpha_u = alpha_ut; u_mu = np.dot(u, alpha_u)
alpha_v = alpha_vt; v_mu = np.dot(v, alpha_v)

#モデルパラメータを生成
beta = betat
theta_u = theta_ut
theta_v = theta_vt

#標準偏差を設定
Sigma = 1

#回帰モデルの平均構造
beta_vec = beta[user_id, :]
beta_mu = np.dot(X * beta_vec, np.repeat(1, column))
W = theta_u[user_id, :]
H = theta_v[item_id, :]
WH = np.dot(W * H, np.repeat(1, k))
mu = beta_mu + WH 

In [160]:
##初期値の設定
##階層モデルのパラメータの設定
Cov = 0.01 * np.diag(np.ones(column)); Cov_inv = np.linalg.inv(Cov)
Cov_u = 0.01 * np.diag(np.ones(k)); Cov_u_inv = np.linalg.inv(Cov_u)
Cov_v = 0.01 * np.diag(np.ones(k)); Cov_v_inv = np.linalg.inv(Cov_v)

#階層モデルの回帰係数を設定
alpha = np.random.normal(0, 0.1, column*column_u).reshape(column_u, column); alpha_mu = np.dot(u, alpha)
alpha_u = np.random.normal(0, 0.1, k*column_u).reshape(column_u, k); u_mu = np.dot(u, alpha_u)
alpha_v = np.random.normal(0, 0.1, k*column_v).reshape(column_v, k); v_mu = np.dot(v, alpha_v)
alpha_mu = np.dot(u, alpha); u_mu = np.dot(u, alpha_u); v_mu = np.dot(v, alpha_v)

#モデルパラメータを生成
beta = np.dot(u, alpha) + np.random.multivariate_normal(np.repeat(0, column), Cov, hh)
theta_u = np.dot(u, alpha_u) + np.random.multivariate_normal(np.repeat(0, k), Cov_u, hh)
theta_v = np.dot(v, alpha_v) + np.random.multivariate_normal(np.repeat(0, k), Cov_v, item)

#標準偏差を設定
Sigma = 1

##潜在効用から応答変数を生成
#回帰モデルの平均構造
beta_vec = beta[user_id, :]
beta_mu = np.dot(X * beta_vec, np.repeat(1, column))
W = theta_u[user_id, :]
H = theta_v[item_id, :]
WH = np.dot(W * H, np.repeat(1, k))
mu = beta_mu + WH 

In [161]:
##データの設定
#切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

#定数を設定
XX = [i for i in range(hh)]
XX_inv = [i for i in range(hh)]
for i in range(hh):
    index = user_index[i]
    XX[i] = np.dot(X[index, :].T, X[index, :])
    
#階層モデルの定数
uu = np.dot(u.T, u)
inv_uu = np.linalg.inv(np.dot(u.T, u) + ADelta)
vv = np.dot(v.T, v)
inv_vv = np.linalg.inv(np.dot(v.T, v) + ADelta_v)

In [162]:
##対数尤度の基準値
#1パラメータでの対数尤度
LLst = np.sum(y*np.log(np.mean(y))) + np.sum((1-y)*np.log(np.mean(1-y)))

#真値での対数尤度
mu = np.dot(X * betat[user_id, :], np.repeat(1, column)) + np.dot(theta_ut[user_id, : ] * theta_vt[item_id, :], np.repeat(1, k))
Prob = norm.cdf(mu, 0, Sigma)
Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))

In [163]:
##サンプリングの格納用配列
BETA = np.zeros((hh, column, int(R/keep)))
THETA_U = np.zeros((hh, k, int(R/keep)))
THETA_V = np.zeros((item, k, int(R/keep)))
ALPHA = np.zeros((column_u, column, int(R/keep)))
ALPHA_U = np.zeros((column_u, k, int(R/keep)))
ALPHA_V = np.zeros((column_v, k, int(R/keep)))
COV = np.zeros((column, column, int(R/keep)))
COV_U = np.zeros((k, k, int(R/keep)))
COV_V = np.zeros((k, k, int(R/keep)))

In [164]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##切断正規分布から潜在効用を生成
    mu = beta_mu + WH   #モデルの平均構造 
    U = rtnorm(mu, Sigma, a, b, hhpt)   #潜在効用を生成
    
    ##ユーザー別に回帰ベクトルをサンプリング
    #応答変数の設定
    er = U - WH
    
    for i in range(hh):
        #データの抽出
        index = user_index[i]
        er_y = er[index]
        x = X[index, :]
        
        #回帰ベクトルの事後分布のパラメータ
        Xy = np.dot(x.T, er_y)
        inv_XXV = np.linalg.inv(XX[i] + Cov_inv)
        beta_mu = np.dot(inv_XXV, Xy + np.dot(Cov_inv, alpha_mu[i, :]))   #回帰ベクトルの期待値
        
        #多変量正規分布から回帰ベクトルをサンプリング
        beta[i, :] = np.random.multivariate_normal(beta_mu, inv_XXV, 1)

    #素性ベクトルの平均を更新
    beta_vec = beta[user_id, :]
    beta_mu = np.dot(X * beta_vec, np.repeat(1, column))

    
    ##ユーザーの特徴行列をサンプリング
    #応答変数の設定
    er = U - beta_mu

    for i in range(hh):
        #データの抽出
        index = user_index[i]
        er_u = er[index]
        x = H[index, :]

        #特徴ベクトルの事後分布のパラメータ
        XXU = np.dot(x.T, x); Xy = np.dot(x.T, er_u)
        inv_XXV = np.linalg.inv(XXU + Cov_u_inv)
        w_mu = np.dot(inv_XXV, Xy + np.dot(Cov_u_inv, u_mu[i, :]))   #特徴ベクトルの期待値

        #多変量正規分布から特徴ベクトルをサンプリング
        theta_u[i, :] = np.random.multivariate_normal(w_mu, inv_XXV, 1)

    #ユーザー特徴行列を変換
    W = theta_u[user_id, :]

    ##アイテムの特徴行列をサンプリング
    #応答変数の設定
    er = U - beta_mu

    for j in range(item):
        #データの抽出
        index = item_index[j]
        er_v = er[index]
        x = W[index, :]

        #特徴ベクトルの事後分布のパラメータ
        XXV = np.dot(x.T, x); Xy = np.dot(x.T, er_v)
        inv_XXV = np.linalg.inv(XXV + Cov_v_inv)
        h_mu = np.dot(inv_XXV, Xy + np.dot(Cov_v_inv, v_mu[j, :]))   #特徴ベクトルの期待値

        #多変量正規分布から特徴ベクトルをサンプリング
        theta_v[j, :] = np.random.multivariate_normal(h_mu, inv_XXV, 1)

    #アイテム特徴行列を変換
    H = theta_v[item_id, :]
    WH = np.dot(W * H, np.repeat(1, k))   #行列分解のパラメータを更新

    
    ##階層モデルのパラメータをサンプリング
    #多変量回帰モデルから素性ベクトルの階層モデルのパラメータをサンプリング
    out = rmultireg(beta, u, inv_uu, np.dot(u.T, beta), Cov, ADelta, Deltabar, V, nu, hh, column_u, column)
    alpha = out[0]
    alpha_mu = np.dot(u, alpha)
    Cov = np.diag(np.diag(out[1]))
    Cov_inv = np.linalg.inv(Cov)

    #多変量回帰モデルからユーザー特徴行列の階層モデルのパラメータをサンプリング
    out = rmultireg(theta_u, u, inv_uu, np.dot(u.T, theta_u), Cov_u, ADelta, Deltabar_u, V1, nu1, hh, column_u, k)
    alpha_u = out[0]
    u_mu = np.dot(u, alpha_u)
    Cov_u = np.diag(np.diag(out[1]))
    Cov_u_inv = np.linalg.inv(Cov_u)

    #多変量回帰モデルからユーザー特徴行列の階層モデルのパラメータをサンプリング
    out = rmultireg(theta_v, v, inv_vv, np.dot(v.T, theta_v), Cov_v, ADelta_v, Deltabar_v, V2, nu2, item, column_v, k)
    alpha_v = out[0]
    v_mu = np.dot(v, alpha_v)
    Cov_v = np.diag(np.diag(out[1]))
    Cov_v_inv = np.linalg.inv(Cov_v)

    
    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[:, :, mkeep] = beta
        THETA_U[:, :, mkeep] = theta_u
        THETA_V[:, :, mkeep] = theta_v
        ALPHA[:, :, mkeep] = alpha
        ALPHA_U[:, :, mkeep] = alpha_u
        ALPHA_V[:, :, mkeep] = alpha_v
        COV[:, :, mkeep] = Cov
        COV_U[:, :, mkeep] = Cov_u
        COV_V[:, :, mkeep] = Cov_v

    if rp%disp==0:
        mu = beta_mu + WH   #モデルの平均構造 
        Prob = norm.cdf(mu, 0, Sigma)
        Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
        LL = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
        print(rp)
        print(np.round(np.array((LL, LLbest, LLst)), 1))
        print(np.round(np.vstack((alpha[0, :], alphat[0, :])), 3))
        print(np.round(np.vstack((np.diag(Cov_u), np.diag(Cov_ut))), 3))

0
[-538628.3 -245319.9 -603232.1]
[[-0.069 -0.131 -0.005 -0.209  0.009 -0.043  0.008  0.012  0.013]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.011 0.011 0.011 0.011 0.011 0.011 0.011 0.011 0.011 0.011]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
10
[-322403.9 -245319.9 -603232.1]
[[-0.476 -0.034  0.144 -0.145 -0.25   0.265  0.044  0.179 -0.203]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.022 0.025 0.027 0.024 0.023 0.023 0.022 0.039 0.022 0.024]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
20
[-288342.  -245319.9 -603232.1]
[[-0.743  0.071  0.381 -0.115 -0.343  0.386  0.099  0.286 -0.326]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.031 0.039 0.043 0.032 0.033 0.037 0.035 0.051 0.037 0.032]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
30
[-273541.6 -245319.9 -603232.1]
[[-0.915  0.17   0.51  -0.113 -0.414  0.452  0.121  0.326 -0.353]
 [ 0.441  0.203

280
[-240931.9 -245319.9 -603232.1]
[[-1.142  0.208  0.617 -0.164 -0.583  0.495  0.402  0.664 -0.262]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.1   0.086 0.091 0.189 0.096 0.126 0.191 0.102 0.117 0.085]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
290
[-241199.4 -245319.9 -603232.1]
[[-1.139  0.166  0.675 -0.219 -0.586  0.554  0.358  0.593 -0.243]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.099 0.091 0.088 0.192 0.104 0.125 0.192 0.097 0.108 0.08 ]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
300
[-240433.6 -245319.9 -603232.1]
[[-1.164  0.21   0.685 -0.165 -0.569  0.489  0.349  0.567 -0.246]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.107 0.095 0.09  0.203 0.102 0.122 0.198 0.097 0.112 0.082]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
310
[-239872.6 -245319.9 -603232.1]
[[-1.15   0.223  0.715 -0.175 -0.577  0.557  0.345  0.661 -0.306]
 [ 0.441  

560
[-240391.7 -245319.9 -603232.1]
[[-1.038  0.27   0.717 -0.129 -0.557  0.492  0.341  0.595 -0.347]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.102 0.125 0.097 0.191 0.12  0.117 0.228 0.095 0.089 0.092]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
570
[-239893.1 -245319.9 -603232.1]
[[-1.046  0.233  0.75  -0.148 -0.588  0.552  0.295  0.543 -0.324]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.101 0.128 0.096 0.189 0.119 0.12  0.219 0.098 0.085 0.095]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
580
[-239918.1 -245319.9 -603232.1]
[[-0.998  0.209  0.767 -0.192 -0.548  0.538  0.315  0.563 -0.379]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.104 0.128 0.098 0.182 0.128 0.119 0.225 0.097 0.083 0.093]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
590
[-240254.7 -245319.9 -603232.1]
[[-1.012  0.251  0.799 -0.12  -0.563  0.534  0.295  0.563 -0.372]
 [ 0.441  

840
[-240424.3 -245319.9 -603232.1]
[[-0.996  0.168  0.747 -0.151 -0.502  0.526  0.38   0.631 -0.272]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.075 0.109 0.091 0.211 0.16  0.115 0.235 0.1   0.077 0.108]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
850
[-240161.1 -245319.9 -603232.1]
[[-0.937  0.184  0.67  -0.163 -0.503  0.544  0.367  0.674 -0.254]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.076 0.1   0.089 0.211 0.165 0.113 0.234 0.104 0.084 0.114]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
860
[-240436.2 -245319.9 -603232.1]
[[-0.916  0.185  0.776 -0.179 -0.564  0.556  0.322  0.589 -0.278]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.074 0.103 0.088 0.206 0.172 0.113 0.232 0.102 0.078 0.112]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
870
[-240350.6 -245319.9 -603232.1]
[[-0.949  0.177  0.643 -0.166 -0.52   0.493  0.311  0.58  -0.3  ]
 [ 0.441  

1120
[-240215.1 -245319.9 -603232.1]
[[-0.902  0.194  0.737 -0.131 -0.563  0.526  0.33   0.614 -0.255]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.08  0.119 0.104 0.21  0.197 0.118 0.236 0.104 0.076 0.128]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1130
[-240213.4 -245319.9 -603232.1]
[[-0.894  0.182  0.679 -0.151 -0.527  0.474  0.306  0.594 -0.274]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.083 0.12  0.105 0.209 0.209 0.118 0.231 0.103 0.078 0.127]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1140
[-240378.6 -245319.9 -603232.1]
[[-0.849  0.106  0.715 -0.184 -0.561  0.542  0.304  0.58  -0.317]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.086 0.126 0.104 0.199 0.213 0.118 0.241 0.109 0.076 0.127]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1150
[-239856.9 -245319.9 -603232.1]
[[-0.922  0.184  0.722 -0.148 -0.585  0.549  0.315  0.594 -0.277]
 [ 0.4

1400
[-239741.2 -245319.9 -603232.1]
[[-0.973  0.235  0.788 -0.181 -0.59   0.556  0.368  0.682 -0.299]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.081 0.106 0.09  0.22  0.219 0.1   0.208 0.109 0.079 0.142]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1410
[-239973.  -245319.9 -603232.1]
[[-0.992  0.217  0.728 -0.165 -0.595  0.528  0.4    0.661 -0.274]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.079 0.113 0.092 0.232 0.225 0.102 0.203 0.11  0.073 0.137]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1420
[-240381.3 -245319.9 -603232.1]
[[-0.98   0.268  0.7   -0.184 -0.59   0.535  0.426  0.669 -0.236]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.082 0.114 0.091 0.238 0.223 0.103 0.223 0.105 0.076 0.135]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1430
[-240612.2 -245319.9 -603232.1]
[[-0.91   0.28   0.718 -0.116 -0.621  0.455  0.391  0.569 -0.188]
 [ 0.4

1680
[-240349.2 -245319.9 -603232.1]
[[-0.845  0.182  0.685 -0.125 -0.538  0.519  0.339  0.549 -0.286]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.072 0.11  0.101 0.245 0.232 0.122 0.245 0.116 0.085 0.171]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1690
[-239564.1 -245319.9 -603232.1]
[[-0.834  0.157  0.691 -0.146 -0.541  0.492  0.352  0.578 -0.298]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.071 0.115 0.102 0.238 0.241 0.121 0.249 0.119 0.082 0.174]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1700
[-240160.9 -245319.9 -603232.1]
[[-0.826  0.143  0.693 -0.182 -0.583  0.493  0.381  0.644 -0.238]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.068 0.117 0.103 0.23  0.24  0.12  0.261 0.117 0.076 0.175]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1710
[-239639.4 -245319.9 -603232.1]
[[-0.914  0.186  0.743 -0.178 -0.586  0.511  0.325  0.587 -0.263]
 [ 0.4

1960
[-240062.9 -245319.9 -603232.1]
[[-0.76   0.25   0.689 -0.12  -0.536  0.551  0.272  0.529 -0.375]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.078 0.111 0.099 0.242 0.267 0.127 0.26  0.128 0.08  0.156]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1970
[-240396.1 -245319.9 -603232.1]
[[-0.824  0.233  0.785 -0.106 -0.518  0.543  0.309  0.573 -0.345]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.074 0.119 0.097 0.242 0.265 0.129 0.263 0.127 0.082 0.151]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1980
[-240312.3 -245319.9 -603232.1]
[[-0.756  0.139  0.823 -0.166 -0.587  0.577  0.307  0.5   -0.295]
 [ 0.441  0.203  0.663 -0.171 -0.581  0.469  0.395  0.625 -0.234]]
[[0.075 0.117 0.1   0.236 0.28  0.129 0.259 0.126 0.076 0.164]
 [0.033 0.06  0.185 0.041 0.196 0.19  0.106 0.144 0.1   0.155]]
1990
[-239754.8 -245319.9 -603232.1]
[[-0.799  0.193  0.778 -0.127 -0.551  0.538  0.3    0.501 -0.367]
 [ 0.4